# Searching for optimal locations for Cloud Kitchens in Mumbai

Project is divided into 3 stages,
- A. Generating Datasets
- B. Search Space Creation
- C. Optimal Location and Location Set Identification
- D. Cuisine recommendations for best locations

###### Approach:
Our analysis is based on computing the distance betweent to location coordinates, and optimising the search using this are the key parameter.

Distance is defined as the sum of the x(or W-E) and y (or N-S) intercept.

The analysis progresses with first rating each venue location as follows,
    rating = (minimum service time for a given set of city centres)/(service time of a location for given set of city centres)

This data will be used to train a model that can generate similar ratings for any point using the following three attributes,
- Number of city centres within serviceable distance
- Average distance between location and the city centres
- Average potential serviceable population

Then we generate a search space of points using the city centre location coordinates, and find points with the best ratings.

It is assumed that these points represent locations that have the maximum potential serviceable population for given set of city centres possible.

After that top locations are sorted and searched for a combination of best locations that serve the maximum proportion of population of the city centres.

Once we have obtained the list of best locations, we simply check the venue categories available at all city centres near our best locations.

We use the Foursquare location data to compute the average venue category at each city centre, and then use the population of the city centres to compute the weighted average for a best location.

###### Key comments
- We have two datasets for city centers, but one lacks population data, hence it is automatically dropped from further analysis
- Still we have utilised that data to generate venue locations from the Foursquare API as it provides more results
- Venue category or cuisine reccomendations are based Foursquare location data
- Functions for all computations will be available in related notebooks



### D. Cuisine Recommender

The recommendation engine considers the presence of a type of venue category/cuisine near the city centre as being analogous to the population showing preference for it.

### D.1. Importing libraries

In [1]:
import numpy as np
import pandas as pd

### D.2. Downloading set of best locations

This set of locations was generated in Part 3.

In [2]:
best_location = pd.read_csv('best_locations_solutions.csv').to_numpy()
best_locations_cords_set = pd.read_csv('best_locations_cords_set.csv').to_numpy()

In [3]:
city_centers = pd.read_csv('m_ward_data.csv').to_numpy()
venue_list = pd.read_csv('m_venue_data.csv').to_numpy()

In [4]:
venue_list[0]

array([0, 'Merwans Cake shop', 19.119300215885477, 72.84541776016009,
       'Bakery'], dtype=object)

In [5]:
best_locations_cords_set[0]

array([0, '[19.1373666, 72.9442834]'], dtype=object)

### D.3. Function definitions

In [6]:
# function to generate list of cuisines/venue types

def cuisine_list_gen(venue_list):
    
    cuisine_list = []
    
    for venue in venue_list:
        
        item = venue[4]
            
        if (item in cuisine_list) != True:
            cuisine_list.append(item)
    
    return cuisine_list

In [7]:
# function to compute distance between two points

# assuming most streets are parallel to either north-south axis or east-west axis
# thus distance is computed as sum of absolute distances

# scale used is 1 degree difference is equal to 111 KM in either axis

def distance(cord1, cord2):
    
    x = (cord1[0]-cord2[0])*111
    
    y = (cord1[1]-cord2[1])*111
    
    if x*y > 0:
        if x > 0:
            return x + y
        else:
            return -1*(x + y)
    else:
        if x > 0:
            return x + (-1 * y)
        else:
            return (-1*x) + y

In [8]:
# function to construct the cuisine recommender's dataset

def cuisine_rec_data_gen(best_locations_cords_set, city_centers, venue_list, distance_limit = 15, initial_distance_limit = 2.5):
    
    best_location_details = []
    
    # adding venue coordinates
    for i in range(len(best_locations_cords_set)):
        
        cords = best_locations_cords_set[i][1].replace("[","").replace("]","").split(',')
        
        best_location_details.append([[float(cords[0]), float(cords[1])]])
    
    # adding venue location details to each location
    for i in range(len(best_location_details)):
        
        city_center_loc = []
        
        map_point = best_location_details[i][0]
        
        for city_centre in city_centers: 

            city_centre_cords = [float(city_centre[8]),float(city_centre[9])]

            city_centre_distance = distance(map_point, city_centre_cords)

            if city_centre_distance <= (distance_limit-initial_distance_limit) and city_centre_distance > 0:
                
                city_center_loc.append([city_centre_cords,int(city_centre[4].replace(",",""))])
                
        best_location_details[i].append(city_center_loc)
    
    # creating cuisine list for each location weighted by each population centers
    
    cc_cuisine_map = cc_cuisine_map_gen(city_centers, venue_list, distance_limit, initial_distance_limit)
    
    cuisine_list = cuisine_list_gen(venue_list)
    
    for i in range(len(best_location_details)):
        
        tot_pop = 0
        
        cuisine_loc_dict = {}
        
        # creating cuisine listing for each location
        for cuisine in cuisine_list:
            
            cuisine_loc_dict[cuisine] = 0
        
        # computing total population of city centers within range
        for cc in best_location_details[i][1]:
            
            tot_pop += cc[1]
        
        # computing total cuisine served at city center within range
        for cc in best_location_details[i][1]:
            
            for j in range(len(cc_cuisine_map)):
                
                if cc[0] == cc_cuisine_map[j][0]:
                    
                    for key in cc_cuisine_map[j][1].keys():
                        
                        cuisine_loc_dict[key] += cc_cuisine_map[j][1][key]*cc[1]/tot_pop
            
        best_location_details[i].append(cuisine_loc_dict)
        
    # returning the top 5 ,bottom 5, and 5 absent cuisines for each locations
    
    best_location_cuisines = []
    
    for i in range(len(best_location_details)):
        
        top5 = []
        
        bot5 = []
        
        absent = []
        
        for key in best_location_details[i][2]:
            
            if best_location_details[i][2][key] == 0:
                
                absent.append(key)
                
        for count in range(5):
            
            top_val = 0
            top_key = ''
            
            bot_val = best_location_details[i][2]['Bakery']
            bot_key = 'Bakery'
            
            for key in best_location_details[i][2]:
                
                if best_location_details[i][2][key] > top_val and (key in top5) != True:
                    top_val = best_location_details[i][2][key]
                    top_key = key
                
                if best_location_details[i][2][key] < bot_val and best_location_details[i][2][key] > 0 and (key in bot5) != True:
                    bot_val = best_location_details[i][2][key]
                    bot_key = key
            
            top5.append(top_key)
            bot5.append(bot_key)            

        # adding info
        best_location_cuisines.append([best_location_details[i][0],{'Top 5':top5,'Bottom 5':bot5,'Absent':absent}])
                    
        
    return best_location_cuisines

In [9]:
# function to create city centre wise cuisine map

def cc_cuisine_map_gen(city_centers, venue_list, distance_limit = 15, initial_distance_limit = 2.5):
    
    cuisine_list = cuisine_list_gen(venue_list)
    
    cc_cuisine_map = []
    
    for city_centre in city_centers:
        
        city_centre_cords = [float(city_centre[8]),float(city_centre[9])]
        
        cc_cuisine_dict = {}
        
        for cuisine in cuisine_list:
            
            cc_cuisine_dict[cuisine] = 0

        for venue in venue_list:
        
            venue_cords = [venue[2],venue[3]]

            city_centre_distance = distance(venue_cords, city_centre_cords)

            if city_centre_distance <= (distance_limit-initial_distance_limit) and city_centre_distance > 0:

                cc_cuisine_dict[venue[4]] += 1 
        
        cc_cuisine_map.append([city_centre_cords,cc_cuisine_dict])
            
            
    return cc_cuisine_map

### D.4. Generating cuisine recommendations

List of unique venue category/cuisine

In [10]:
cuisine_list = cuisine_list_gen(venue_list)
print("Count of unique venue category/cuisinecuisine_list: {}".format(len(cuisine_list)))

Count of unique venue category/cuisinecuisine_list: 70


In [11]:
# generating list of Top 5, Bottom 5, absent cuisines

best_location_cuisines = cuisine_rec_data_gen(best_locations_cords_set, 
                                              city_centers, venue_list, 
                                              distance_limit = 15, 
                                              initial_distance_limit = 2.5)

In [12]:
len(best_location_cuisines)

4

In [13]:
# printing top 5 cuisines for each location

for i in range(len(best_location_cuisines)):
    
    print(best_location_cuisines[i][1]['Top 5'])

['Indian Restaurant', 'Café', 'Fast Food Restaurant', 'Chinese Restaurant', 'Restaurant']
['Indian Restaurant', 'Café', 'Fast Food Restaurant', 'Chinese Restaurant', 'Restaurant']
['Indian Restaurant', 'Café', 'Fast Food Restaurant', 'Chinese Restaurant', 'Restaurant']
['Indian Restaurant', 'Café', 'Chinese Restaurant', 'Fast Food Restaurant', 'Bakery']


In [14]:
# printing bottom 5 cuisines for each location

for i in range(len(best_location_cuisines)):
    
    print(best_location_cuisines[i][1]['Bottom 5'])

['German Restaurant', 'Modern European Restaurant', 'Fish & Chips Shop', 'Brazilian Restaurant', 'Multicuisine Indian Restaurant']
['Dhaba', 'German Restaurant', 'Multicuisine Indian Restaurant', 'Gluten-free Restaurant', 'English Restaurant']
['English Restaurant', 'Chaat Place', 'Burrito Place', 'Dumpling Restaurant', 'Parsi Restaurant']
['North Indian Restaurant', 'Creperie', 'Afghan Restaurant', 'Comfort Food Restaurant', 'Mughlai Restaurant']


In [15]:
# printing absent cuisines for each location

for i in range(len(best_location_cuisines)):
    
    print(best_location_cuisines[i][1]['Absent'])

['Dhaba', 'English Restaurant', 'Cafeteria', 'Molecular Gastronomy Restaurant', 'Parsi Restaurant', 'Steakhouse', 'Chaat Place', 'Portuguese Restaurant']
['Cafeteria', 'Molecular Gastronomy Restaurant', 'Parsi Restaurant', 'Steakhouse', 'Chaat Place', 'Portuguese Restaurant']
['Dhaba']
['Dhaba', 'English Restaurant', 'Burrito Place', 'Dumpling Restaurant']
